In [1]:
import pyro
import torch

import numpy as np

## Data generating model

In [484]:
def model(num_time_steps):
    
    """
    Toy Gillespie model implemented in pyro. Used to test different observational scenerios.
    """
    
    batch = torch.tensor([[0,0,0]] * num_time_steps)
    
    ## Latent model parameters. Interested in infering rate
    with pyro.plate("latent_rates"):
        #r1 = pyro.param("rate", torch.tensor(.5),
        #                     constraint=pyro.distributions.constraints.positive)
        r1 = pyro.sample("r1", pyro.distributions.Normal(.5, .01))
    
    with pyro.plate("data"):
    #for i in range(len(data)):    
        ## Initial model setup
        #with pyro.plate("starting_state"):
        s1_temp = pyro.sample("s1_start", pyro.distributions.Normal(10, .00001))#, obs = data[0, i, 0])
        s2_temp = pyro.sample("s2_start", pyro.distributions.Normal(10, .00001))#, obs = data[0, i, 1])
        s3_temp = pyro.sample("s3_start", pyro.distributions.Normal(10, .00001))#, obs = data[0, i, 2])
            # t_prev = pyro.sample("time_start", pyro.distributions.Delta(torch.tensor(0)), obs = data[0, 3])
        batch[0] = torch.tensor([s1_temp, s2_temp, s3_temp])
        ## Time series sampling and updates
        for t in range(1, num_time_steps):

            ## Gillespie sampling
            #with pyro.plate("latent_discrete"):
            sample = pyro.sample("sample_{0}".format(str(t)), 
                                     pyro.distributions.Categorical(
                                         torch.tensor([s1_temp * r1, 
                                                       s2_temp * .1, 
                                                       s3_temp * .9])))#,
                                     #infer={"enumerate": "parallel"})

            ## Update species
            update = np.zeros(3)
            update[sample] = 1
            s1_temp = pyro.sample("s1_{0}".format(str(t)), 
                               pyro.distributions.Normal(s1_temp + update[0], .00001))#, 
                               #obs = data[t, i, 0])
            s2_temp = pyro.sample("s2_{0}".format(str(t)), 
                               pyro.distributions.Normal(s2_temp + update[1], .00001))#, 
                               #obs = data[t, i, 1])
            s3_temp = pyro.sample("s3_{0}".format(str(t)), 
                               pyro.distributions.Normal(s3_temp + update[2], .00001))#, 
                               #obs = data[t, i, 2])
            batch[t] = torch.tensor([s1_temp, s2_temp, s3_temp])
            ## Update time
            # t_temp =  pyro.sample("t_{0}_temp".format(str(t)), 
            #                       pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
            # t_t =  pyro.sample("t_{0}".format(str(t)), pyro.distributions.Normal(t_prev + t_temp, .00001),
            #                    obs = data[t,3])

            # s1_prev = s1_t
            # s2_prev = s2_t
            # s3_prev = s3_t
            # t_prev = t_t
            
    return batch


In [485]:
data = list()
for x in range(100):
    temp = model(100)
    data.append(temp)
data = torch.stack(data)

## Incorrectly specified model

In [386]:
s1_temp = torch.tensor([4,12,12])
s2_temp = torch.tensor([4,12,12])
s3_temp = torch.tensor([4,12,12])

In [384]:
with pyro.plate("loop", size = len(s1_temp)):
    pyro.sample("test", pyro.distributions.Categorical(torch.tensor([torch.tensor([4,12,12]) * .5,
                                                  torch.tensor([4,12,12]) * .5,
                                                  torch.tensor([4,12,12]) * .5])))

ValueError: only one element tensors can be converted to Python scalars

In [390]:
data.shape[1]

10

In [575]:
def model(data):
    
    ## Latent model parameters. Interested in infering rate
    #with pyro.plate("latent_rates"):
        
    ## Rate parameter
    rate = pyro.param("rate", torch.tensor(.99),
                     constraint=pyro.distributions.constraints.positive)

    r1 = pyro.sample("r1", pyro.distributions.Normal(rate, .01))

    ## starting points
    s1_start = pyro.sample("s1_start", pyro.distributions.Normal(10, .00001))
    s2_start = pyro.sample("s2_start", pyro.distributions.Normal(10, .00001))
    s3_start = pyro.sample("s3_start", pyro.distributions.Normal(10, .00001))

    with pyro.plate("loop", data.shape[0]) as i:
        
        for t in range(data.shape[1]):

            ## Gillespie sampling
            #with pyro.plate("latent_discrete"):
            sample = pyro.sample("sample_{0}".format(str(t)), 
                                     pyro.distributions.Categorical(
                                         torch.tensor([s1_start * r1, 
                                                       s2_start * .1, 
                                                       s3_start * .9])).to_event(1))#,
                                     #infer={"enumerate": "parallel"})

            ## Update species
            update = np.zeros(3)
            update[sample] = 1
            s1_temp = pyro.sample("s1_{0}".format(str(t)), 
                               pyro.distributions.Normal(s1_start + update[0], .00001).to_event(1), 
                               obs = data[i, t, 0])
            s2_temp = pyro.sample("s2_{0}".format(str(t)), 
                               pyro.distributions.Normal(s2_start + update[1], .00001).to_event(1), 
                               obs = data[i, t, 1])
            s3_temp = pyro.sample("s3_{0}".format(str(t)), 
                               pyro.distributions.Normal(s3_start + update[2], .00001).to_event(1), 
                               obs = data[i, t, 2])
            # batch[t] = torch.tensor([s1_temp, s2_temp, s3_temp])
            ## Update time
            # t_temp =  pyro.sample("t_{0}_temp".format(str(t)), 
            #                       pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
            # t_t =  pyro.sample("t_{0}".format(str(t)), pyro.distributions.Normal(t_prev + t_temp, .00001),
            #                    obs = data[t,3])

            s1_start = s1_temp
            s2_start = s2_temp
            s3_start = s3_temp
            # t_prev = t_t
            
    return s1_start


In [576]:
model(data)

ValueError: Expected reinterpreted_batch_ndims <= len(base_distribution.batch_shape), actual 1 vs 0

In [439]:
nuts_kernel = pyro.infer.NUTS(model)
mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=500)
mcmc.run(data)
samples = mcmc.get_samples()

Warmup:   0%|                                            | 0/1000 [00:00, ?it/s]

ValueError: number of dimensions must be within [0, 32]
  Trace Shapes:                                                                      
   Param Sites:                                                                      
           rate                                                                      
  Sample Sites:                                                                      
        r1 dist                                                                     |
          value                                                                     |
  s1_start dist                                                                     |
          value                                                                     |
  s2_start dist                                                                     |
          value                                                                     |
  s3_start dist                                                                     

Warmup:   6%|▏ | 64/1000 [00:15, 36.76it/s, step size=2.96e-08, acc. prob=0.743]

In [435]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
        r1     -1.08      0.00     -1.08     -1.08     -1.08     11.09      1.04
  s1_start      9.50      0.00      9.50      9.50      9.50      0.50       nan
  s2_start      9.00      0.00      9.00      9.00      9.00      0.58      1.00
  s3_start      9.00      0.00      9.00      9.00      9.00      0.50       nan

Number of divergences: 446


In [588]:
def model(data):
    
    ### Latent model parameters. Interested in infering rate
    #with pyro.plate("latent_rates"):
        
    ## Rate parameter
    #rate = pyro.param("rate", torch.tensor(.99))#,
#                     constraint=pyro.distributions.constraints.positive)
    rate = pyro.param("rate", torch.tensor(.99),
                     constraint=pyro.distributions.constraints.positive)
    r1 = pyro.sample("r1", pyro.distributions.Normal(rate, .01))

    ## starting points
    s1_start = pyro.sample("s1_start", pyro.distributions.Normal(10, .00001))
    s2_start = pyro.sample("s2_start", pyro.distributions.Normal(10, .00001))
    s3_start = pyro.sample("s3_start", pyro.distributions.Normal(10, .00001))

    for i in range(data.shape[0]):
        
        for t in range(data.shape[1]):

            ## Gillespie sampling
            with pyro.plate("latent_discrete"):
                sample = pyro.sample("sample_{0}_{1}".format(str(t), str(i)), 
                                         pyro.distributions.Categorical(
                                             torch.tensor([s1_start * r1, 
                                                           s2_start * .1, 
                                                           s3_start * .9])),
                                         infer={"enumerate": "parallel"})

            ## Update species
            with pyro.plate("update_species"):
                update = np.zeros(3)
                update[sample] = 1

                s1_temp = pyro.sample("s1_{0}_{1}".format(str(t), str(i)), 
                                   pyro.distributions.Normal(s1_start + update[0], .00001), 
                                   obs = data[i, t, 0])
                s2_temp = pyro.sample("s2_{0}_{1}".format(str(t), str(i)), 
                                   pyro.distributions.Normal(s2_start + update[1], .00001), 
                                   obs = data[i, t, 1])
                s3_temp = pyro.sample("s3_{0}_{1}".format(str(t), str(i)), 
                                   pyro.distributions.Normal(s3_start + update[2], .00001), 
                                   obs = data[i, t, 2])
            # batch[t] = torch.tensor([s1_temp, s2_temp, s3_temp])
            ## Update time
            # t_temp =  pyro.sample("t_{0}_temp".format(str(t)), 
            #                       pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
            # t_t =  pyro.sample("t_{0}".format(str(t)), pyro.distributions.Normal(t_prev + t_temp, .00001),
            #                    obs = data[t,3])

            s1_start = s1_temp
            s2_start = s2_temp
            s3_start = s3_temp
            # t_prev = t_t
            
    return s1_start


In [560]:
def guide(data):
    
    ## Latent model parameters. Interested in infering rate
    #with pyro.plate("latent_rates"):
        
    ## Rate parameter
    rate = pyro.param("rate", torch.tensor(.99),
                     constraint=pyro.distributions.constraints.positive)

    pyro.sample("r1", pyro.distributions.Normal(rate, .001))

    ## starting points
    s1_start = pyro.sample("s1_start", pyro.distributions.Normal(10, .00001))
    s2_start = pyro.sample("s2_start", pyro.distributions.Normal(10, .00001))
    s3_start = pyro.sample("s3_start", pyro.distributions.Normal(10, .00001))

    for i in range(data.shape[0]):
        
        for t in range(data.shape[1]):

            ## Gillespie sampling
            #with pyro.plate("latent_discrete"):
            pyro.sample("sample_{0}_{1}".format(str(t), str(i)), 
                                     pyro.distributions.Categorical(
                                         torch.tensor([s1_start * r1, 
                                                       s2_start * .1, 
                                                       s3_start * .9])))#,
                                    # infer={"enumerate": "parallel"})

            

In [555]:
guide = pyro.infer.autoguide.AutoNormal(pyro.poutine.block(model, expose=["r1"]))

pyro.clear_param_store()
elbo = pyro.infer.TraceEnum_ELBO(max_plate_nesting=1)
elbo.loss(model, guide, data)

ValueError: number of dimensions must be within [0, 32]
   Trace Shapes:                                                                    
    Param Sites:                                                                    
   Sample Sites:                                                                    
         r1 dist                                                                   |
           value                                                                   |
   s1_start dist                                                                   |
           value                                                                   |
   s2_start dist                                                                   |
           value                                                                   |
   s3_start dist                                                                   |
           value                                                                   |
 sample_0

In [590]:
# set up the optimizer
adam_params = {"lr": 0.05, "betas": (0.90, 0.999)}
optimizer = pyro.optim.Adam(adam_params)
guide = pyro.infer.autoguide.AutoDelta(pyro.poutine.block(model, 
            hide=["sample_{0}_{1}".format(str(x), str(y)) for x in range(100) for y in range(100)]))#, hide=['sample_0_0'])
#guide = pyro.infer.autoguide.AutoNormal(pyro.poutine.block(model, expose=["r1"]))

# setup the inference algorithm
svi = pyro.infer.SVI(model, guide, optimizer, loss=pyro.infer.Trace_ELBO())

n_steps = 100
pyro.clear_param_store()
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 10 == 0:
        print('.', end='')


/Users/kohler.d/opt/anaconda3/lib/python3.9/site-packages/pyro/util.py:303: UserWarning: Found vars in model but not guide: {'sample_66_12', 'sample_15_7', 'sample_23_67', 'sample_23_4', 'sample_52_41', 'sample_46_89', 'sample_23_69', 'sample_71_74', 'sample_49_48', 'sample_37_85', 'sample_99_26', 'sample_24_28', 'sample_99_93', 'sample_19_65', 'sample_74_92', 'sample_35_15', 'sample_19_66', 'sample_61_36', 'sample_92_72', 'sample_99_41', 'sample_64_88', 'sample_85_54', 'sample_36_6', 'sample_71_38', 'sample_13_5', 'sample_97_99', 'sample_69_54', 'sample_26_92', 'sample_93_58', 'sample_65_15', 'sample_89_93', 'sample_53_96', 'sample_18_10', 'sample_99_72', 'sample_45_64', 'sample_19_7', 'sample_80_43', 'sample_59_78', 'sample_44_80', 'sample_42_16', 'sample_7_95', 'sample_90_24', 'sample_52_62', 'sample_33_43', 'sample_92_13', 'sample_69_36', 'sample_17_32', 'sample_16_7', 'sample_79_14', 'sample_52_26', 'sample_90_79', 'sample_79_59', 'sample_91_54', 'sample_56_50', 'sample_24_73', 's

..........

In [ ]:
pyro.param("rate").item()